#### Basic Field Camera GIRF measurement
Make a simple GIRF sequence for a field camera that measures a series of slightly different chirp waveforms.  It cycles through all channels, and should take 9 minutes to run.

In [2]:
import pickle
from pathlib import Path

import numpy as np
from helpers import (
    PSeq_Base,
    PSeq_SkopeTrigger,
    PSeq_Moments3D,
    PSeq_Params,
    PSeq_TestWave_Chirp,
)

In [3]:
# Setup base sequernce parameters
params = PSeq_Params(channels=('x', 'y', 'z'), max_grad=32, max_slew=140)

# This holds the full sequence
pseq0 = PSeq_Base(params)

# Slice excitation
pseq_excite = PSeq_SkopeTrigger(params)

# Setup chirp test waveforms
pseq_test = PSeq_TestWave_Chirp(params)

# Refocus the test waveforms and any other spatial encoding
pseq_refocus = PSeq_Moments3D(
    params,
    start_areas=(None, None, pseq_test.all_areas),
    end_areas=(0, 0, 0),
    start_polarities=(1, 1, (-1, 0, 1)),
)

# Other parameters:
N_av = 10
TR = 500e-3

In [4]:
all_meas = []

pseq0.reinit_seq()
for i_av in range(N_av):
    for channels in [("y", "z", "x"), ("x", "z", "y"), ("x", "y", "z")]:
        params.channels = channels
        for wave_idx in range(pseq_test.N_waves):
            # 0 polarity can be removed to save time, but it may be useful in some types of processing
            for polarity in [-1, 0, 1]:  
                
                pseq0.channels = channels

                pseq0.track_time = 0
                pseq0 += pseq_excite.build_blocks()
                pseq0 += pseq_test.build_blocks(idx=wave_idx, polarity=polarity)
                pseq0 += pseq_refocus.build_blocks(idx2=wave_idx, start_pol2=polarity)

                pseq0.add_delay(TR-pseq0.track_time)
                params.increment_rf_spoiling()
                
                all_meas.append(
                    {'channels': channels,
                    'wave_idx': wave_idx,
                    'polarity': polarity,
                    'i_av': i_av,
                    },
                )

In [5]:
seq_name = f'cam_girf_v1_{N_av}av'
print(f'======== {seq_name} ========')

pseq0.seq.set_definition(key="Name", value=seq_name)
pseq0.seq.set_definition(key="FOV", value=[50e-3, 50e-3, 50e-3])  

pseq0.check()

save_res = {'all_meas': all_meas}
Path('./seqs/').mkdir(parents=True, exist_ok=True)
pseq0.seq.write(f'./seqs/{seq_name}.seq')
with Path(f'./seqs/{seq_name}.pickle').open('wb') as handle:
    pickle.dump(save_res, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
print(' * SAVED! *')

======== cam_girf_v1_10av ========
Seq Time: 0:09:00
Timing check passed successfully
 * SAVED! *
